In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langgraph.graph import END, StateGraph

/Users/pritaman/Documents/ML/LangChain/LangGraph/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from models import GraphState

In [4]:
workflow = StateGraph(GraphState)

In [5]:
from main import generate, grade_documents, retrieve, transform_query, web_search


workflow.add_node("web_search", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae
workflow.add_node("transform_query", transform_query) # transform_query

/Users/pritaman/Documents/ML/LangChain/LangGraph/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from main import route_question

In [7]:
workflow.set_conditional_entry_point(
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)

In [8]:
workflow

In [9]:
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")

In [10]:
from main import decide_to_generate


workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)

In [11]:
workflow.add_edge("transform_query", "retrieve")

In [12]:
from main import grade_generation_v_documents_and_question


workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

In [13]:
app = workflow.compile()

In [14]:
inputs = {"question": "What are the types of agent memory?"}

In [16]:
from pprint import pprint

In [17]:
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value ["generation"])

{'question': 'What are the types of agent memory?'}
---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('The types of agent memory are sensory memory, short-term memory (STM) or '
 'working memory, and long-term memory (LTM). LTM has two subtypes: '
 'explicit/declarative memory and implicit/procedural memory.')


In [1]:
state = {'question': 'What are the types of agent memory?'}

question = state["question"]

In [3]:
from adaptive_rag import question_router


question_router().invoke({"question": question})

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ALshRFfValNa82jFlhyS76OO', 'function': {'arguments': 'const RouteQuery: functions.RouteQuery = ({ datasource }) => {\n  if (datasource === "vector_store") {\n    return "The types of agent memory are episodic memory, semantic memory, and procedural memory.";\n  } else {\n    return { datasource: "websearch" };\n  }\n};', 'name': 'RouteQuery'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 149, 'total_tokens': 210}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-375a0cd7-50e9-4067-9f1b-51f

In [ ]:
from adaptive_rag import decider_template

source = decider_template(user_query=question)

In [4]:
from langchain_openai import AzureChatOpenAI


llm = AzureChatOpenAI(
    openai_api_key='7425866287b14892ac27f27785acd412',
    openai_api_version="2024-02-15-preview",
    azure_deployment="chat35turbo",
    azure_endpoint="https://novaplayground.openai.azure.com/",
    temperature=0
)

In [14]:
from models import RouteQuery
from langchain.prompts import ChatPromptTemplate

structured_llm_router = llm.with_structured_output(RouteQuery, include_raw=True)

system="""
You are an expert in routing user questions to a vector store or web search. 
The vector store contains documents related to agents, prompt engineering and adversarial attacks.
Use the vector-store for questions on these topics. Other use web-search.
Do not return a function or other content. Only return RouteQuery datasource as 'vector_store' or 'websearch' in curly braces
"""

route_template = ChatPromptTemplate.from_messages(
    [('system', system),
    ('human', '{question}')]
)

router = route_template | structured_llm_router

In [15]:
router.invoke({'question': question})

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uerWN5Xusp85tPauvCALYVwZ', 'function': {'arguments': '{\n  datasource: "vectorstore"\n}', 'name': 'RouteQuery'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 148, 'total_tokens': 157}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-5e73ecc8-ac48-4eca-9977-868e51c8c158-0', invalid_tool_calls=[{'name': 'RouteQuery', 'args': '{\n  datasource: "vectorstore"\n}', 'id': 'call_uerWN5Xusp85tPauvCALYVwZ', 'error': 'Function RouteQuery arguments:\n\n{\n  datasource: "vectorstore"\n}\n\nare not vali

In [16]:
from adaptive_rag import preprocess_json_string


output = router.invoke({'question': question})

parsed_path = RouteQuery.parse_raw(
    preprocess_json_string(
        output["raw"].additional_kwargs["tool_calls"][0]['function']['arguments']))


ValidationError: 1 validation error for RouteQuery
datasource
  unexpected value; permitted: 'vector_store', 'websearch' (type=value_error.const; given=vectorstore; permitted=('vector_store', 'websearch'))

In [ ]:
parsed_path.datasource

'vector_store'